In [1]:
MODEL_NAME = "intfloat/multilingual-e5-small"

WIKIPEDIA_DS = "singletongue/wikipedia-utils"
WIKIPEDIA_DS_NAME = "passages-c400-jawiki-20230403"
DS_NAME = "hotchpotch/wikipedia-passages-jawiki-embeddings"
INDEX_NAME = f"faiss_indexes/{WIKIPEDIA_DS_NAME}/multilingual-e5-small-passage/index_IVF2048_PQ96.faiss"

In [2]:
from datasets.download import DownloadManager
from datasets import load_dataset  # type: ignore

ds = load_dataset(path=WIKIPEDIA_DS, name=WIKIPEDIA_DS_NAME, split="train")
dm = DownloadManager()
index_local_path = dm.download(
    f"https://huggingface.co/datasets/{DS_NAME}/resolve/main/{INDEX_NAME}"
)
f"https://huggingface.co/datasets/{DS_NAME}/resolve/main/{INDEX_NAME}"

/Users/hotchpotch/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'https://huggingface.co/datasets/hotchpotch/wikipedia-passages-jawiki-embeddings/resolve/main/faiss_indexes/passages-c400-jawiki-20230403/multilingual-e5-small-passage/index_IVF2048_PQ96.faiss'

In [3]:
ds

Dataset({
    features: ['id', 'pageid', 'revid', 'title', 'section', 'text'],
    num_rows: 5555583
})

In [4]:
from sentence_transformers import SentenceTransformer

MODEL = SentenceTransformer(MODEL_NAME, device="cpu")
MODEL.max_seq_length = 512

In [5]:
import faiss

index = faiss.read_index(index_local_path)
index.ntotal

5555583

In [6]:
index.nprobe = 128

In [7]:
import numpy as np

prefix = ""

if "-e5-" in INDEX_NAME:
    prefix = "query: "


def to_emb(text, prefix=prefix) -> np.ndarray:
    return MODEL.encode([prefix + text], normalize_embeddings=True)  # type: ignore


emb = to_emb("フリーレンの声優は誰?")
emb.shape

(1, 384)

In [8]:
import time


def faiss_search_by_text(text, ds, faiss_index=index, top_k=10):
    emb = to_emb(text)
    start_time = time.time()
    D, I = faiss_index.search(emb, top_k)
    end_time = time.time()
    print(f"Search took {end_time - start_time:.2f} seconds")
    results = []
    for idx, (id, d) in enumerate(zip(I[0], D[0])):
        if idx >= 0:
            id = int(id)
            results.append((ds[id], d))
    return results


# q = 'フリーレンの声優は誰？'
# q = 'フリーレンの悪役は?'
q = "宝川温泉が東の横綱、湯原温泉が西の横綱と呼ばれる、温泉地ではおなじみの野外に設置されたお風呂を何というでしょう?"
# q = '日本で最大の湖は?'

for data, score in faiss_search_by_text(q, ds):
    print(score, data["title"], data["text"])
    # print(list(data.keys()))

Search took 0.01 seconds
0.17603306 露天風呂 露天風呂番付(ろてんぶろばんづけ)は、温泉番付の一種であり、露天風呂が有名な温泉地を番付したものである。作成されたのは1981年で、野口冬人の手で編集された。露天風呂番付における西の横綱は湯原温泉の「砂湯」、東の横綱は宝川温泉である。湯原温泉には大きな露天風呂番付のモニュメントが掲示されており、温泉地のシンボルとなっている。
0.2231432 水上温泉郷 詳しくは、主人公であるローマ帝国のテルマエ設計技師ルシウス(演:阿部寛)がタイムスリップして出没する“平たい顔族(日本人もしくは大和民族のこと)”の露天風呂が、汪泉閣で最も大きい露天風呂「子宝の湯」である。宝川(たからがわ)は、利根川上流域の支流の一つであり、一級河川である。群馬県利根郡みなかみ町藤原と新潟県南魚沼市藤原との県境に位置する中央分水界を成す大烏帽子山の、南麓を源流域として発し、おおよそ南東方向へ流れて、宝台樹山(水上宝台樹スキー場を擁する山)の北西麓に広がる扇状地にある奥利根橋付近 で利根川本流 に合流する。上述のとおり、「宝川温泉」の名の由来になっている。宝泉峡(ほうせんきょう)は、宝川の下流域にある渓谷。宝川温泉から800mほど上流にある。総延長約2km。
0.23655552 湯原温泉 現在、利用されている温泉はその一部だけでほとんどは未利用のまま川に流されている。河原に遊歩道及び駐車場が整備されている。基本は湯治、保養、療養向けであるが、場所柄、山陰・山陽の行楽拠点としても重宝され、落ち着いた湯街の風情は観光地の売りのひとつである。湯原温泉を代表する名所で湯原ダムの下にある巨大な共同露天風呂「砂噴き湯:砂湯」。川底から砂を噴きながら温泉が湧いていることから砂噴き湯(砂湯)の名称で呼ばれるようになった。足下噴出源泉露天風呂である。旅行作家の会代表の野口冬人が、日本温泉協会発行の「温泉」紙上に1977年(昭和52年)に発表した露天風呂番付において西の横綱とされる。川の中にあるお風呂ながら住民により管理され無料で開放されていることが横綱の評価となった。 この砂湯は湯原温泉の古代からのお風呂の様子を唯一残す物で市の文化財として指定されている。ここでの入浴は混浴である。
0.2401702 水上温泉郷 昭和の温泉評論家・